In [ ]:
from TimeLLMProcessors import FixedModelTimeLLMProcessor, UpdatingModelTimeLLMProcessor

In [ ]:
from ARIMAProcessors import FixedARIMAProcessor, UpdatingARIMAProcessor

In [ ]:
from LSTMProcessors import FixedModelLSTMProcessor, UpdatingModelLSTMProcessor

In [ ]:
import pandas as pd

In [ ]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)

In [ ]:
FAProcessor = FixedARIMAProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])
FLProcessor = FixedModelLSTMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])
FTProcessor = FixedModelTimeLLMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])

In [ ]:
FAProcessor.create_training_dfs(value_col = "value")
FLProcessor.create_training_dfs(value_col = "value")
FTProcessor.create_training_dfs(value_col = "value")

In [ ]:
FAProcessor.create_fixed_model(h = 4, freq = "W-SAT", season_length = 52, model_name = "testing", level = [])

In [ ]:
def config_LSTM(trial):
    return {
        "encoder_hidden_size": trial.suggest_categorical("encoder_hidden_size", [100, 200, 300]),
          "encoder_n_layers": trial.suggest_categorical("encoder_n_layers", [1, 2, 3]),
          "context_size": trial.suggest_categorical("context_size", [5, 10, 20, 30, 40, 50]),
          "decoder_hidden_size": trial.suggest_categorical("decoder_hidden_size", [64, 128, 256, 512]),
          "max_steps": 1000,
          "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32, 64]),
          "input_size": trial.suggest_categorical("input_size", [16, 64, 256])
          }

In [ ]:
FLProcessor.create_fixed_model(h = 4, freq = "W-SAT", model_name = "test_model", level = [], config = config_LSTM)

In [ ]:
prompt = "This dataset contains data on weekly flu hospitalizations in the United States. There is a yearly (52 weeks) seasonality"
FTProcessor.create_fixed_model(h = 4, freq = "W-SAT", model_name = "test_model", prompt = prompt)

In [ ]:
FAProcessor.create_display_df()
FAProcessor.create_metrics()
FAProcessor.create_metrics_df()

FLProcessor.create_display_df()
FLProcessor.calculate_metrics()
FLProcessor.create_metrics_df()

FTProcessor.create_display_df()
FTProcessor.calculate_metrics()
FTProcessor.create_metrics_df()

In [ ]:
FAProcessor.metrics_df

In [ ]:
FLProcessor.metrics_df

In [ ]:
FTProcessor.metrics_df